<a href="https://colab.research.google.com/github/supakornkr/RoadToDE-Workshop/blob/main/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install PyMySql

In [16]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.8 MB/s 


In [5]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### Connect DB

In [18]:
import pymysql
class config:
  HOST = os.getenv('MYSQL_HOST')
  PORT = int(os.getenv('MYSQL_PORT'))
  USER = os.getenv('MYSQL_USER')
  PASSWORD = os.getenv('MYSQL_PASSWORD')

3306